# Requirements

In [1]:
import os
import hickle as hkl
import h5py
import numpy as np
import requests
import time
import concurrent.futures
import pandas as pd

---
# save links to a file

In [4]:
df = pd.read_csv('Datasets/Air Conditioners.csv')
img_urls = df['image'].values
np.savetxt('images/img_urls.txt', img_urls, fmt='%s')

In [5]:
with open('images/img_urls.txt', 'r') as url_list:
    img_urls = [str(line) for line in url_list.readlines() if line[-2] != 'f']

---
# download images

### Store images in the dataset

In [49]:
def store(dataset, meta_set, img, img_name):
    dataset[img_name] = img
    meta_set[img_name] = img_name

In [50]:
def download_image(dataset, meta_set, img_url):
    img_bytes = requests.get(img_url).content
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    img = np.frombuffer(img_bytes, np.uint8)
    
    # # store img
    # store(dataset, meta_set, img, img_name)

In [0]:
size = len(img_urls)
# []
imgs = [] * 
with h5py.File('images/images.h5', 'w') as hf:
    dataset = hf.create_dataset("images", np.shape(size))
    meta_set = hf.create_dataset("meta", np.shape(size))
    for url in img_urls:
        download_image(dataset, meta_set, url)